In [ ]:
import polars as pl

from src.util.constants import DATA_PATH

In [ ]:
df_all = pl.read_parquet(DATA_PATH / 'df_all.parquet')

In [ ]:
df_all.head()

In [ ]:
df_time_count = df_all.group_by('date_id', maintain_order=True).agg(pl.col('time_id').unique().count())
df_time_count.plot.line(x='date_id', y='time_id').show()

We reach the maximum number of time_ids before halftime.

In [ ]:
df_all['symbol_id'].unique().count()

There are 39 unique symbols in the dataset.

In [ ]:
df_available_symbols = df_all.group_by('date_id', maintain_order=True).agg(pl.col('symbol_id').unique()).explode('symbol_id')
df_symbol_availability = df_available_symbols.group_by('symbol_id').agg(pl.col('date_id').min().alias('first_date'), pl.col('date_id').max().alias('last_date')).sort('symbol_id')

df_symbol_availability.describe()

Once a symbol is available, it is available for all time_ids and for every subsequent date_id (not shown).

The last symbols come available at date 1063, which marks the 62% date quantile.



In [ ]:
df_null_count = df_all.group_by(['date_id'], maintain_order=True).agg(pl.all().exclude(['date_id', 'time_id']).null_count() / pl.col('date_id').count())

In [ ]:
# plot a line for each feature in df_null_count for which the maximum count is larger than zero:
for col in df_null_count.columns[1:]:
    if df_null_count[col].max() > 0:
        df_null_count.plot.line(x='date_id', y=col).show()

All features become usually available before halftime.

Most features can be null.

In [ ]:
df_all[round(len(df_all)*.7), :]

If we use 10% of the data each for feature selection, validation, and testing, we can train on data up to date 1314, mwaning we get at least 250 days for each symbol and for each feature.